In [1]:
!pip install nltk
!pip install scikit-learn
!pip install sklearn
!pip install numpy

In [2]:
from nltk.tokenize import word_tokenize 
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from math import sqrt
from math import pi
from math import exp
from random import randrange

In [3]:
data=pd.read_csv(r'C:\Users\Sai charan\Documents\Machine Learning\emails.csv')

In [4]:
data.drop(['Email No.'],axis=1,inplace=True)

# Using inbuilt function

In [5]:
# Y = data['Prediction']
# X = data.drop(['Prediction'], axis=1)
# X = np.array(X)
# Y = np.array(Y)

In [6]:
# print(X.shape)
# print(Y.shape)

In [7]:
# X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.13)
# print(X_train.shape)
# print(Y_train.shape)

In [8]:
# clf=MultinomialNB()
# clf.fit(X_train,Y_train)
# y_pred = clf.predict(X_test)
# accuracy_score(y_pred,Y_test)

## Without using inbuilt function

In [10]:
# Utility functions 
def mean(numbers):
 return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
 avg = mean(numbers)
 variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
 return sqrt(variance)
 
def summarize_dataset(dataset):
 summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
 del(summaries[-1])
 return summaries

def calculate_gaussian_probability(x, mean, stdev):
 exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
 return (1 / (sqrt(2 * pi) * stdev)) * exponent
 
def get_class_probabilities(summaries, row):
 total_rows = sum([summaries[label][0][2] for label in summaries])
 probabilities = dict()
 for class_value, class_summaries in summaries.items():
     probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
     for i in range(len(class_summaries)):
         mean, stdev, _ = class_summaries[i]
     probabilities[class_value] *= calculate_gaussian_probability(row[i], mean, stdev)
 return probabilities

############################################################################

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
 dataset_split = list()
 dataset_copy = list(dataset)
 fold_size = int(len(dataset) / n_folds)
 for _ in range(n_folds):
     fold = list()
     while len(fold) < fold_size:
         index = randrange(len(dataset_copy))
         fold.append(dataset_copy.pop(index))
     dataset_split.append(fold)
 return dataset_split
 
def accuracy_metric(actual, predicted):
 correct = 0
 for i in range(len(actual)):
     if actual[i] == predicted[i]:
         correct += 1
 return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, n_folds):
 folds = cross_validation_split(dataset, n_folds)
 scores = list()
 # training in batches
 for fold in folds:
     train_set = folds
     train_set.remove(fold)
 train_set = sum(train_set, [])
 test_set = list()
 for row in fold:
     row_copy = list(row)
     test_set.append(row_copy)
     row_copy[-1] = None
 predicted = naive_bayes(train_set, test_set)
 actual = [row[-1] for row in fold]
 accuracy = accuracy_metric(actual, predicted)
 scores.append(accuracy)
 return scores
 
# Split the dataset by class values, returns a dictionary
# 0s will contain list of non spam emails and 1 will contain list of spam emails
def separate_by_class(dataset):
 separated = dict()
 for i in range(len(dataset)):
     vector = dataset[i]
     class_value = vector[-1]
     if (class_value not in separated):
         separated[class_value] = list()
     separated[class_value].append(vector)
 return separated

 
# Split dataset by class then calculate statistics for each row -> Getting the mean and standard deviation
def summarize_by_class(dataset):
 separated = separate_by_class(dataset)
 summaries = dict()
 for class_value, rows in separated.items():
     summaries[class_value] = summarize_dataset(rows)
 return summaries
 
# Predict the class for a given row
def predict(summaries, row):
 probabilities = get_class_probabilities(summaries, row)
 best_label, best_prob = None, -1
 for class_value, probability in probabilities.items():
     if best_label is None or probability > best_prob:
         best_prob = probability
         best_label = class_value
 return best_label
 
# Naive Bayes Algorithm
def naive_bayes(train, test):
 summarize = summarize_by_class(train)
 predictions = list()
 for row in test:
     output = predict(summarize, row)
     predictions.append(output)
 return(predictions)
 
## Starting... 
dataset = np.array(data)
n_folds = 2
scores = evaluate_algorithm(dataset, n_folds)
print(f"Accuracy: {(sum(scores)/float(len(scores)))} %")

Accuracy: 71.46171693735499 %
